# cleaning ast from project file

In [13]:
import json

In [14]:
project_data = {}
with open('data/parsing-test/project.json', 'r') as f:
    project_data = json.load(f)

In [15]:
targets = project_data['targets']

In [16]:
#targets[0] is always the stag
sprite = targets[1]

In [17]:
blocks = sprite['blocks']

In [25]:
# all opcodes present in this sprite
[block['opcode'] for bid, block in blocks.items()]

['event_whenflagclicked',
 'control_repeat',
 'control_if',
 'event_whenthisspriteclicked',
 'event_whenbroadcastreceived',
 'motion_turnright',
 'motion_movesteps',
 'motion_goto',
 'motion_goto_menu',
 'motion_goto',
 'motion_goto_menu',
 'motion_goto',
 'motion_goto_menu',
 'operator_gt',
 'operator_random',
 'looks_say',
 'looks_thinkforsecs',
 'event_whenflagclicked',
 'control_if_else',
 'control_if_else',
 'motion_movesteps',
 'motion_turnleft',
 'motion_glideto',
 'motion_glideto_menu',
 'operator_gt',
 'operator_gt',
 'operator_random',
 'operator_random']

In [20]:
def get_top_blocks(blocks, is_list=False):
    if is_list:
        return [block for bid, block in blocks.items() if 'topLevel' in list(block) and block['topLevel']]
    return {bid: block for bid,block in blocks.items() if 'topLevel' in list(block) and block['topLevel']}

In [24]:
def traverse(blocks):
    
    """
    traverses the blocks dictionary by continually traversing the 'next' property of each block
    returns a list of top to bottom paths
    """
    
    # 1. Grab the top blocks as reference points
    tops = get_top_blocks(blocks)
    topids = [bid for bid,block in tops.items()]
    
    # Store each stack
    stacks = []
        
    # start at each top block and traverse down by seeing each next block
    for tid in topids:
        stack = []
        currid = tid
        while(currid != None):
            stack.append(blocks[currid]['opcode'])
            currid = blocks[currid]['next']
        stacks.append(stack)
    
    return stacks

In [22]:
paths = traverse(blocks)

In [23]:
paths

[['event_whenflagclicked', 'control_repeat'],
 ['event_whenthisspriteclicked', 'control_if'],
 ['event_whenbroadcastreceived', 'motion_turnright'],
 ['motion_movesteps', 'motion_goto', 'looks_thinkforsecs', 'looks_say'],
 ['event_whenflagclicked', 'control_if_else']]

In [ ]:
# how woudl you manually build up a tree?
